In [ ]:
# default_exp circuit

# Circuit

> Representation of quantum circuits.

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# export
from qsam.nbtools import patch
from collections.abc import MutableSequence

A circuit is a list of ticks. A tick consists of gates associated to qubits. We can have one-qubit gates which are applied to single gate locations and two-qubit gates which are applied to an ordered (control,target)-qubit pair. For simplicity, we implement this as a list of dictionaries. Each tick is represented by a dictionary, a *tick*tionary :), which holds the `gate_symbol` as key and the set of `qubits` we want to apply these gates to as values. Qubits can be either *integers* for single-qubit locations or *tuples* for two-qubit locations.

In [ ]:
#export
class Circuit(MutableSequence):

    """Representation of a quantum circuit"""
    def __init__(self, ticks=None):
        self._ticks = ticks if ticks else []
        
    def __getitem__(self, tick_index):
        return self._ticks[tick_index]
    
    def __setitem__(self, tick_index, tick):
        self._ticks[tick_index] = tick
        
    def __delitem__(self, tick_index):
        del self._ticks[tick_index]
        
    def __len__(self):
        return len(self._ticks)
    
    def insert(self, tick_index, tick):
        self._ticks.insert(tick_index, tick)
    
    def __str__(self):
        str_list = []
        for i, tick in enumerate(self._ticks):
            str_list.append(f"{i}: {str(tick)}")
        return "\n".join(str_list)
    
    def __repr__(self):
        return self.__str__()

Having defined the base structure of a circuit let's do some tests

In [ ]:
c = Circuit([{'H': {1,2,3}}])
assert c._ticks == [{'H': {1,2,3}}]

In [ ]:
c.append({'X': {0,1}, 'CNOT':{(2,3)}})
assert c._ticks == [{'H': {1,2,3}}, {'X': {0,1}, 'CNOT':{(2,3)}}]

In order to use our `Circuit` class for simulations it is very useful to define a couple of more advanced functions. First, we can see from the `Simulator` class that we need to know the **number of qubits** present in a circuit. If we'd only had single qubit gates we could simply take the cardinality of the union of all `values` per tick. Since we also have two-qubit gates we must `unpack` (i.e. flatten) those tuples first and then take the union. Let's define a helper function for that.

In [ ]:
#export
def unpack(seq):
    if isinstance(seq, (tuple,set,list)):
        yield from (x for y in seq for x in unpack(y))
    elif isinstance(seq, dict):
        yield from (x for v in seq.values() for y in v for x in unpack(y))
    else:
        yield seq

Let's check if `unpack` does what we want by calling it on our above defined small circuit:

In [ ]:
print(c._ticks)
print(list(unpack(c._ticks)))

[{'H': {1, 2, 3}}, {'X': {0, 1}, 'CNOT': {(2, 3)}}]
[1, 2, 3, 0, 1, 2, 3]


That's looking good. We collect all qubits on which we apply gates in this circuit in a single list. Now, we want to define two properties to return the set of unique `qubits` and the number of those, `n_qubits`.

In [ ]:
# export
Circuit.qubits = property(lambda self: set(unpack(self._ticks)))
Circuit.n_qubits = property(lambda self: len(self.qubits))

Let's test our new properties:

In [ ]:
print(c.qubits)
print(c.n_qubits)

{0, 1, 2, 3}
4


Furthermore, we need some way to query the circuit for locations of certain sets of gates, for example all two-qubit gates. This becomes important when we want to apply multi-parameter noise models, i.e. different physical error rates for single-qubit gates, two-qubit gates, measurements, initialization etc. As output of this function we want a list of (tick_index,qubit(s))-tuples for which the applied gates are inside the specified gate set.

In [ ]:
#export
@patch(Circuit)
def partition(self, gateset):
    [(tick_idx,) for tick_idx,tick in enumerate(self._ticks) for gate,qbs in tick.items() if gate.upper() in gateset]